### Analysis Given Data

In [22]:
# use pickle load the data
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
from solve_m_height_cpp import solveMHeight_glpk
import time

from data_loader_saver import load_data, save_data


In [23]:
m_h_datapath = "../data/project/DS-15-samples_mHeights"
n_k_m_datapath = "../data/project/DS-15-samples_n_k_m_P"

In [24]:
n_k_m_P, m_heights = load_data(n_k_m_datapath, m_h_datapath)

# quick sanity checks
print(f'm_heights type: {type(m_heights)}, length: {len(m_heights)}')
print(f'n_k_m type: {type(n_k_m_P)}, length: {len(n_k_m_P)}')

print(f'First 5 m_heights: \n{m_heights[0]}')
print(f'First 5 n_k_m: \n{n_k_m_P[0]}')

print(f'num samples:{len(m_heights)}, {len(n_k_m_P)}')

k = n_k_m_P[0][1]
#create identity matrix of size k by k
I_k = np.eye(k)

print(f'Identity matrix shape: {I_k.shape}')

#construct G matrix as I_k | M
M = n_k_m_P[0][3]
G = np.hstack((I_k, M))

print(f'G matrix shape: {G.shape}')
print(f'G matrix: \n{G}')

m = n_k_m_P[0][2]
m_h_calc = solveMHeight_glpk(G, m)[0]

print(f'Calculated m_height: \n{m_h_calc}')
print(f'Given m_height: \n{m_heights[0]}')

m_heights type: <class 'list'>, length: 32087
n_k_m type: <class 'list'>, length: 32087
First 5 m_heights: 
11.472222222222221
First 5 n_k_m: 
[9, 4, 2, array([[ 1, -2,  2, -1,  4],
       [ 3,  0, -2,  1,  3],
       [ 0,  4,  4,  4,  3],
       [-3, -1,  4,  1, -4]])]
num samples:32087, 32087
Identity matrix shape: (4, 4)
G matrix shape: (4, 9)
G matrix: 
[[ 1.  0.  0.  0.  1. -2.  2. -1.  4.]
 [ 0.  1.  0.  0.  3.  0. -2.  1.  3.]
 [ 0.  0.  1.  0.  0.  4.  4.  4.  3.]
 [ 0.  0.  0.  1. -3. -1.  4.  1. -4.]]
Calculated m_height: 
11.472222222222223
Given m_height: 
11.472222222222221


In [25]:
from collections import Counter

combo_counter = Counter()
combo_stats = {}
for sample in n_k_m_P:
    n_k_m = tuple(sample[:3])
    G_values = np.asarray(sample[3])
    combo_counter[n_k_m] += 1

    g_min = G_values.min()
    g_max = G_values.max()
    if n_k_m not in combo_stats:
        combo_stats[n_k_m] = {'min': g_min, 'max': g_max}
    else:
        combo_stats[n_k_m]['min'] = min(combo_stats[n_k_m]['min'], g_min)
        combo_stats[n_k_m]['max'] = max(combo_stats[n_k_m]['max'], g_max)

unique_n_k_m = list(combo_counter.keys())
unique_n_k_m_count = list(combo_counter.values())

print(f'Number of unique (n, k, m) combinations: {len(unique_n_k_m)}')
print(f'Unique (n, k, m) combinations: {unique_n_k_m}')
print('Counts and G stats per (n, k, m):')
for combo in unique_n_k_m:
    count = combo_counter[combo]
    g_min = combo_stats[combo]['min']
    g_max = combo_stats[combo]['max']
    print(f'{combo}: count={count}, G_min={g_min}, G_max={g_max}')


Number of unique (n, k, m) combinations: 9
Unique (n, k, m) combinations: [(9, 4, 2), (9, 4, 3), (9, 4, 4), (9, 4, 5), (9, 5, 2), (9, 5, 3), (9, 5, 4), (9, 6, 2), (9, 6, 3)]
Counts and G stats per (n, k, m):
(9, 4, 2): count=2677, G_min=-100, G_max=100
(9, 4, 3): count=2588, G_min=-100, G_max=100
(9, 4, 4): count=2376, G_min=-100, G_max=100
(9, 4, 5): count=1494, G_min=-100, G_max=100
(9, 5, 2): count=3579, G_min=-100, G_max=100
(9, 5, 3): count=3286, G_min=-100, G_max=100
(9, 5, 4): count=2002, G_min=-100, G_max=100
(9, 6, 2): count=9336, G_min=-100, G_max=100
(9, 6, 3): count=4749, G_min=-100, G_max=100


### Generate Training Set

Since we don't have enough data for each n_k_m pair, we will generate more data for each unique m_k_m pair so that we have can train a seperate neural network for each n_k_m pair. 


In [26]:
from run_gen_data_set import gen_training_samples

start_time = time.time()
n_k_m_G_samples, m_h_samples = gen_training_samples(9, 4, 2, 100)
end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 3.2431046962738037 seconds


In [19]:
print(f'{n_k_m_G_samples[0], m_h_samples[0]}')

((9, 4, 2, array([[ 27,  94,  11,   9, -19],
       [ 38, -48,  -8, -28, -68],
       [-39, -32, -85, -34, -30],
       [-65,  20, -18,  89,   7]])), 126.52074345407694)
